In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [5]:
indie_df = pd.read_csv('Data/indiepygo_ready.csv')
indie_df.head(10)

,category,year_end,month_end,day_end,funded_percent,tagline,title,goal_usd,australia,canada,switzerland,denmark,western_europe,great_britain,hong_kong,norway,sweden,singapore,united_states,education,productivity,energy_greentech,wellness,comics,fashion_wearables,video_games,photography,tv_shows,dance_theater,phones_accessories,audio,film,transportation,art,environment,writing_publishing,music,travel_outdoors,health_fitness,tabletop_games,home,local_business,food_beverage,culture,human_rights,podcasts_vlogs,camera_gear,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,is_success,friday,monday,tuesday,wednesday,thursday,saturday,sunday
0,Transportation,2010,5,12,16.80,Pure electric motorcycle proves a powerful alt...,Join the Electric Revolution!!!,5000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Human Rights,2010,7,2,20.83,Send Me to Haiti...I'm needed there!,Relief Trip to Haiti,1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Human Rights,2010,7,10,16.67,Haiti Relief Mission to Leogane,"Out To Reach Leogane, Haiti 2010",1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Photography,2010,10,9,25.00,My third book on Fezziwig Press.,The Transpersonal Papers: 1861-2010,2000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,Human Rights,2011,1,12,0.65,Homeless Veterans Transitional Housing Develop...,Homeless Veterans need a Hand UP not Hand Out!,55000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,Camera Gear,2011,1,1,69.91,Because he fed his to a shark,Help Buy Scotland A Camera,2200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
6,Photography,2011,1,21,15.50,Touring the US,BEAUTY SHOP CULTURE EXHIBITION,5000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
7,Photography,2011,3,28,1.51,Images from the forgotten camps of Bangladesh:...,Photos from the other side,4700.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
8,Human Rights,2011,3,31,399.83,Representing The Face of Gender Equality Today,This Is What A Feminist Looks Like - Feminist ...,3500.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
9,Transportation,2011,5,1,20.00,Ride for Childhood Cancer,Don's Hat Rack Annual Motorcycle Rally,2000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
#remove the % in the funded column
indie_df['funded_percent'] = indie_df['funded_percent'].str.replace('%', '').astype(float)

In [6]:
indie_df.loc[indie_df['funded_percent'] == 80, 'is_success'] = 1
indie_df

,category,year_end,month_end,day_end,funded_percent,tagline,title,goal_usd,australia,canada,switzerland,denmark,western_europe,great_britain,hong_kong,norway,sweden,singapore,united_states,education,productivity,energy_greentech,wellness,comics,fashion_wearables,video_games,photography,tv_shows,dance_theater,phones_accessories,audio,film,transportation,art,environment,writing_publishing,music,travel_outdoors,health_fitness,tabletop_games,home,local_business,food_beverage,culture,human_rights,podcasts_vlogs,camera_gear,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,is_success,friday,monday,tuesday,wednesday,thursday,saturday,sunday
0,Transportation,2010,5,12,16.80,Pure electric motorcycle proves a powerful alt...,Join the Electric Revolution!!!,5000.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Human Rights,2010,7,2,20.83,Send Me to Haiti...I'm needed there!,Relief Trip to Haiti,1200.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Human Rights,2010,7,10,16.67,Haiti Relief Mission to Leogane,"Out To Reach Leogane, Haiti 2010",1200.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Photography,2010,10,9,25.00,My third book on Fezziwig Press.,The Transpersonal Papers: 1861-2010,2000.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,Human Rights,2011,1,12,0.65,Homeless Veterans Transitional Housing Develop...,Homeless Veterans need a Hand UP not Hand Out!,55000.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,Film,2020,8,30,14.47,Trans filmmaker making queer documentaries. It...,Totally Gay Productions,9338.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
20627,Music,2020,8,16,2.17,"Help us finance Every Stone, our first EP!",Lady Crow debut EP,3697.5021,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
20628,Fashion & Wearables,2020,9,15,2.43,"Benefits Save Venice, helps to protect Venice'...","JOSA, the ultimate Venice-Inspired bracelet",3346.0297,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
20629,Comics,2020,8,1,5.00,Hi,My Campaign Title,2000.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
indie_df['is_success'] = (indie_df['funded_percent'] > 80).astype(int)

In [ ]:
indie_df['is_success'].value_counts()

In [7]:
df = pd.read_csv('Data/indiepygo_ready.csv')

In [ ]:
df.info()

In [ ]:
from datetime import datetime
df['date_launch'] = pd.to_datetime(df['date_launch'])
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['day_of_week'] = df['date_launch'].apply(lambda x: day_names[x.weekday()])

In [ ]:
df['day_of_week'].value_counts()

In [ ]:
df.info()

In [ ]:
#dummifying 
day_dummies = pd.get_dummies(df['day_of_week'])

In [ ]:
df = pd.concat([df, day_dummies], axis=1)

In [ ]:
df.info()

In [ ]:
df.columns = df.columns.str.lower()
df.info()

In [ ]:
df.to_csv('indiegogo_2.csv', index=False) #save with column day of week

In [ ]:
days_df = df[['friday', 'monday', 'tuesday', 'wednesday', 'thursday', 'saturday', 'sunday']]

In [ ]:
type(days_df)

In [ ]:
indie_df = pd.concat([indie_df, days_df], axis=1)
indie_df.info()

In [ ]:
indie_df.to_csv('indiepygo_ready.csv', index=False) #save with columns day of week

In [ ]:
indie_text_df = indie_df[['title', 'tagline', 'is_success']]
indie_text_df

In [ ]:
indie_text_df['text'] = indie_text_df['title'] + ' ' + indie_text_df['tagline']
indie_text_df['text']

In [ ]:
indie_text_df.drop(['title', 'tagline'], axis=1, inplace=True)

In [ ]:
indie_text_df.to_csv('indiepygo_text_fornlp.csv', index=False)